#Adversarial examples

#prenom/nom: Julien Vu

### Libraries

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
#ajout librarie torch.nn.functionnal
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

### Chargement des données MNIST

In [ ]:
# load MNIST dataset
def load_mnist(split, batch_size):
  train = True if split == 'train' else False
  dataset = datasets.MNIST("./data", train=split, download=True, transform=transforms.ToTensor())
  return DataLoader(dataset, batch_size=batch_size, shuffle=train)

batch_size = 100
train_loader = load_mnist('train', batch_size)
test_loader = load_mnist('test', batch_size)
# load one batch of images from the training set
dataiter = iter(train_loader)
#next() function returns the next item from the iterator.
images, labels = dataiter.next()
print(images)
print(labels)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

### Implementer la classe ProjectedGradientDescent.

Vous devez implementer la fonction "compute" qui génère et retourne à partir de $(x,y)$ un exemple adversarial $\tilde{x} = x + \delta$ tel que model$(\tilde{x})\neq y$ en utilisant la methode PGD.

In [ ]:
class ProjectedGradientDescent:
  
  def __init__(self, model, eps, alpha, num_iter):
    #initialisation des différents paramètres du problème
     super(ProjectedGradientDescent, self).__init__() 
     self.model=model
     self.eps=eps
     self.alpha=alpha
     self.num_iter=num_iter
  
  def compute(self, x, y):
    """ Construct PGD adversarial pertubration on the examples x."""  
    #Génération d'un exemple adversarial x+delta à partir de (x,y)
    #delta: intialisé avec des zeros 
    x.requires_grad=True
    for i in range(self.num_iter) :  
      #choix de la fonction loss: la cross-entropy
      loss = nn.CrossEntropyLoss(x, y)
      #on fait une dérivation de la fonction loss
      loss.backward()
      delta = x.grad.sign()
      #Donnees de la perturbation delta et fixer les valeurs entre - epsilon et +epsilon
      delta_data = (delta + x.data).clamp(-self.eps,self.eps)
    #la méthode detach() permet de créer un tenseur qui partage le stockage avec un tenseur qui ne nécessite pas de gradient
    return delta_data.detach()



### Construire l'architecture de votre modèle

Vous devez construire un reseau de neurones convolutionnel.

In [ ]:
class ConvModel(torch.nn.Module):
  
  def __init__(self):
    super(ConvModel, self).__init__()
    # spécification de toutes les couches du réseau nécessitant des hyperparamètres
    # On les crée dans la methode __init__. Les connexions entre les couches va être établi en-dessous  dans  la méthode forward(). 
    # les liens entre les couches sont spécifiés par la méthode forward() ci-dessous.
    self.conv1 = nn.Conv2d(in_channels=1, padding=2, out_channels=32, kernel_size=5) # what is the output size?
    print(self.conv1)
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # what is the output size?
    print(self.pool1)
    self.conv2 = nn.Conv2d(in_channels=32, padding=2, out_channels=64, kernel_size=5)# what is the output size?
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # what is the output size?
    self.fc1 = nn.Linear(in_features=64*7*7, out_features=1024) # where is the value passed as the in_features argument coming from?
    self.fc2 = nn.Linear(in_features=1024, out_features=10)
    self.dropout = nn.Dropout(p=0.5)
    
  def forward(self, x):
    # cette méthode précise comment le réseau traite ses entrées - en d'autres termes, elle décrit la connectivité du réseau.
    # première convolution, suivie de la non-linéarité et de la mise en commun des ReLU
    x = self.pool1(F.relu(self.conv1(x)))
    # deuxième convolution, suivie par la non-linéarité et la mise en commun des ReLU
    x = self.pool2(F.relu(self.conv2(x)))
    # remodeler la couche en concaténant tous les éléments et  en supprimant leur disposition "topographique". c est comme un reshape
    x = x.view(-1, 64*7*7)
    # première couche entièrement connectée (avec non-linéarité ReLU) 
    x = F.relu(self.fc1(x))
    #abandon (régularisation)
    x = self.dropout(x)
    # deuxième couche entièrement connectée
    x = self.fc2(x)
    #correspondre toutes les valeurs à l'opération [0,1], de sorte que la sortie finale puisse être interprétée comme 
    # probabilité d'appartenir à chacune des classes de labels.
    x = F.log_softmax(x, dim=1)
    return x
    

In [ ]:
#utilisation du device pour prendre un environnement d execution la gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#reseau de neurones convolutionnels
model = ConvModel()
model.to(device)
#définir l'optimiseur SGD
opt=optim.SGD(model.parameters(), lr=0.1, momentum=0.2) 
#choix de epsilon=3
epsilons=0.1
#choix du alpha
alpha=0.02
# define the attack
#si l'attack n'est  none, nouvelle instance du pgd
pgd=ProjectedGradientDescent(model,epsilons, alpha,150)
#initialisation du criterion
criterion=nn.CrossEntropyLoss()


Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


### Créer la fonction d'entrainement.

On remarque que la fonction "training" prend en argument "attack". Si "attack" est égal à None, l'entrainement se fait sur les données naturelles (non-attaquées). A l'inverse, si "attack" est une instance de classe d'attaque (ex. ProjectedGradientDescent) alors vous devez entrainer votre modèle sur des examples attaqué grace à l'instance "attack".

In [ ]:
def training(model, criterion, optimizer, loader, attack):
  """Function to train the model"""
  # code here ...
  for epoch in range(24):
    for i, data in enumerate(train_loader):
      #si le modèle n est pas attaqué
      if attack == None:
        # get the next minibatch of data découpés en batchs
        inputs,labels = data
        #utiliser cuda pour avoir plus de rapidité du code
        inputs,labels=inputs.cuda(),labels.cuda()
      else:
        #si le modèle est attaqué
        #on fait le pgd
        inputs,labels=pgd.compute(data[0],data[1])

      #générer des valeurs de sortie prédites en appelant la methode forward
      outputs=model.forward(inputs)
      #choix de l'optimiseur
      loss=criterion(outputs,labels)
      #on calcule les gradients automatiquement
      loss.backward()
      opt.step()
      # zero the gradients (this is needed because PyTorch accumulates gradients 
      # #mise à jour des paramètres du réseau
      optimizer.zero_grad()   
    if epoch % 4 == 0: 
        print(f'[Iteration {epoch}] Loss: {loss.item()}')
training(model, criterion, opt, train_loader, None)

[Iteration 0] Loss: 0.003745482536032796
[Iteration 4] Loss: 0.001685173949226737
[Iteration 8] Loss: 3.587732862797566e-05
[Iteration 12] Loss: 0.000133646753965877
[Iteration 16] Loss: 0.00043795446981675923
[Iteration 20] Loss: 0.011640599928796291


### Implémenter la fonction d'évalutation. 

Cette fonction doit renvoyer l'accuracy du modèle. Si attack est égal à None, vous devez renvoyer l'accuracy du modèle testé sur des données naturelle (non-attaquées). Si attack est different de None, vous devez renvoyer l'accuracy du modèle testé sur des données attaquée.

In [ ]:
def eval_model(model, loader, attack=None):
  # load one batch of images from the loader data
  dataiter = iter(loader)
  #next() function returns the next item from the iterator.
  inputs, labels = dataiter.next()
  inputs, labels=inputs.cuda(), labels.cuda()
  outputs=model(inputs)
  num_examples = inputs.size()[0]
  #valeur de sortie prédite
  _, predictions = torch.max(outputs, 1)
  #print(' '.join(('{}'.format(j) for j in labels)))
  #affichage des labels
  print("Ground truth: ", " ".join("{}".format(labels[j]) for j in range(num_examples)))
  #valeurs prédites predictions
  print("Predicted:    ", " ".join("{}".format(predictions[j]) for j in range(num_examples)))
  #calcul de l accuracy
  print("Accuracy: {}%".format(100*(labels==predictions).sum()/num_examples))

#tester de ls accuracy modele non attaquees sur les donnnes test
eval_model(model,test_loader)







    
    


  




Ground truth:  5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6 0 7 6 1 8 7 9 3 9 8 5 9 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0 4 5 6 1 0 0 1 7 1 6 3 0 2 1 1 7 9 0 2 6 7 8 3 9 0 4 6 7 4 6 8 0 7 8 3 1
Predicted:     5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6 0 7 6 1 8 7 9 3 9 8 5 9 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0 4 5 6 1 0 0 1 7 1 6 3 0 2 1 1 7 9 0 2 6 7 8 3 9 0 4 6 7 4 6 8 0 7 8 3 1
Accuracy: 100.0%
